<center><h1>Neural Computation Coursework<br>Group 42

<center><h3><b>Latif Rozaqi (1980172)<br>Latif Rozaqi (1980172)<br>Latif Rozaqi (1980172)<br>Latif Rozaqi (1980172)<br>Latif Rozaqi (1980172)<br>

<h2>
    <b>I. Data</b>
</h2>

<h3> 1.1 Import Libraries and Functions

In [3]:
################################### Libraries #####################################
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import h5py, os
import numpy as np
from torch.utils.data import DataLoader
import torchvision.utils as vutils
from functions import transforms as T
from functions.subsample import MaskFunc
from scipy.io import loadmat
from matplotlib import pyplot as plt
import csv
import time
from skimage.measure import compare_ssim 

################################## Functions #######################################
def show_slices(data, slice_nums, cmap=None): # visualisation
    fig = plt.figure(figsize=(15,10))
    for i, num in enumerate(slice_nums):
        plt.subplot(1, len(slice_nums), i + 1)
        plt.imshow(data[num], cmap=cmap)
        plt.axis('off')

class MRIDataset(DataLoader):
    def __init__(self, data_list, acceleration, center_fraction, use_seed, cplx):
        self.data_list = data_list
        self.acceleration = acceleration
        self.center_fraction = center_fraction
        self.use_seed = use_seed
        self.cplx=cplx

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        subject_id = self.data_list[idx]
        return get_epoch_batch(subject_id, self.acceleration, self.center_fraction, self.use_seed, self.cplx)

def get_epoch_batch(subject_id, acc, center_fract, use_seed=True, cplx=False):
    ''' random select a few slices (batch_size) from each volume'''

    fname, rawdata_name, slice = subject_id  
    Cplx=cplx
    with h5py.File(rawdata_name, 'r') as data:
        rawdata = data['kspace'][slice]
                      
    rawdata=crop_img(rawdata,320,320)
            
    slice_kspace = T.to_tensor(rawdata).unsqueeze(0)
    S, Ny, Nx, ps = slice_kspace.shape
    
    slice_kspace_abs=torch.log(T.complex_abs(slice_kspace) + 1e-9)
    # apply random mask
    shape = np.array(slice_kspace.shape)
    shape2 = np.array(slice_kspace2.shape)
    mask_func = MaskFunc(center_fractions=[center_fract], accelerations=[acc])
    seed = None if not use_seed else tuple(map(ord, fname))
    mask = mask_func(shape, seed)
    mask2 = mask_func(shape2, seed)
      
    # undersample
    masked_kspace = torch.where(mask == 0, torch.Tensor([0]), slice_kspace)
    masked_kspace2 = torch.where(mask2 == 0, torch.Tensor([0]), slice_kspace2)
    masks = mask.repeat(S, Ny, 1, ps)
    masked_kspace_abs=torch.log(T.complex_abs(masked_kspace) + 1e-9)

    img_gt, img_und = T.ifft2(slice_kspace), T.ifft2(masked_kspace)
    # perform data normalization which is important for network to learn useful features
    # during inference there is no ground truth image so use the zero-filled recon to normalize
    norm = T.complex_abs(img_und).max()
    if norm < 1e-6: norm = 1e-6
    
    # normalized data
    img_gt, img_und, rawdata_und = img_gt/norm, img_und/norm, masked_kspace/norm
       
    img_gt=T.center_crop(T.complex_abs(img_gt),[320,320])
    img_und=T.center_crop(T.complex_abs(img_und),[320,320])
    if not Cplx:
        return img_und, img_gt # x is the undersampled image y is the ground truth
    else:
        return masked_kspace2, slice_kspace2

def load_data_path(train_data_path, val_data_path):
    """ Go through each subset (training, validation) and list all 
    the file names, the file paths and the slices of subjects in the training and validation sets 
    """

    data_list = {}
    train_and_val = ['train', 'val']
    data_path = [train_data_path, val_data_path]
      
    for i in range(len(data_path)):

        data_list[train_and_val[i]] = []
        
        which_data_path = data_path[i]
    
        for fname in sorted(os.listdir(which_data_path)):
            
            subject_data_path = os.path.join(which_data_path, fname)
                     
            if not os.path.isfile(subject_data_path): continue 
            
            with h5py.File(subject_data_path, 'r') as data:
                num_slice = data['kspace'].shape[0]
                
            # the first 5 slices are mostly noise so it is better to exlude them
            data_list[train_and_val[i]] += [(fname, subject_data_path, slice) for slice in range(5, num_slice)]
    
    return data_list    

def show_img(data, slice_nums, train_num, cmap=None): # visualisation
    fig = plt.figure(figsize=(15,10))
    for i, num in enumerate(slice_nums):
        plt.subplot(1, len(slice_nums), i + 1)
        plt.imshow(data[train_num,num], cmap=cmap)
        plt.axis('off')
        
def ssim(gt, pred):
    """ Compute Structural Similarity Index Metric (SSIM). """
    return compare_ssim(
        gt.transpose(1, 2, 0), pred.transpose(1, 2, 0), multichannel=True, data_range=gt.max()
    )

<h3> 1.2 Configure the Training Dataset
    <h4> 1.2.1 Split Dataset

<h5>Here we split the Dataset into training and validation data with the proportion of 80:20 randomly with seed generator

In [ ]:
# random number seed generator
SeedGen = 42
# set the path of trainig dataset
data_path_train = '/data/local/NC2019MRI/train'
data_path_val = '/data/local/NC2019MRI/train'
# compute the number of files inside the directory
num_files=len(os.listdir(data_path_train))
index=(np.arange(num_files))
# generate random number for index
np.random.seed(SeedGen)
# shuffle index randomly
np.random.shuffle(index)
# split the train and validate data by 80:20
train_index=np.sort(index[:int(num_files*0.8)])
validate_index=np.sort(index[int(num_files*0.8):])
# create dictionary contains the information about the traindata and validation data from train dataset
data_list = {}
data_list['train']=[]
data_list['validate']=[]
for idx,fname in enumerate(sorted(os.listdir(data_path_train))): 
    subject_data_path = os.path.join(data_path_train, fname)         
    if not os.path.isfile(subject_data_path): continue         
    with h5py.File(subject_data_path, 'r') as data:
            num_slice = data['kspace'].shape[0]
    if len(np.where(train_index==idx)[0]):
        data_list['train']+=[(fname,subject_data_path,slice) for slice in range(5,num_slice)]
    else:
        data_list['validate']+=[(fname,subject_data_path,slice) for slice in range(5,num_slice)]

In [ ]:
batch_size=5 # set training batch size
acc = 8 # set the cartesian undersampling acceleration factor
cen_fract = 0.04
seed = False # random masks for each slice 
num_workers = 0 # data loading is faster using a bigger number for num_workers. 0 means using one cpu to load data
cplx = False # True: choose real-valued image otherwise: complex k-space data
# create data loader for training set. It applies same to validation set as well
train_dataset = MRIDataset(data_list['train'], acceleration=acc, center_fraction=cen_fract, use_seed=seed, cplx=cplx)
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size, num_workers=num_workers) 

<h2>
    <b>II. Network Architecture</b>
</h2>

In [ ]:
# Choose the model: uncomment one of models below
# model='CNN3L'
# model='CNN4L'
# model='UNet'
model='DCGAN'

<h3> 2.1 Convolutional Neural Network (SRCNN)
<h4> 2.1.1) 3 Layers-CNN
<img src="SRCNN.png"
     alt="SRCNN figure"
     style="float: left; margin-right: 10px;" />

In [ ]:
if model=='CNN3L':
    class CNN_3L(nn.Module):
        def __init__(self):
            super(CNN_3L,self).__init__()
            ### Patch Extraction n1=64 c=1 f1=9
            self.block1=nn.Conv2d(1,64,kernel_size=9,padding=2)
            ### Non-Linear Mapping n2=n1=64 f2=1
            self.block2=nn.Conv2d(64,32,kernel_size=1,padding=2)
            ### Image Reconstruction n3=n2=32 f3=5
            self.block3=nn.Conv2d(32,1,kernel_size=5,padding=2)
            self.RELU=nn.ReLU()
        def forward(self,out):
            out=self.block1(out)
            out=self.RELU(out)
            out=self.block2(out)
            out=self.RELU(out)
            out=self.block3(out)
            return out

<h4>2.1.2) 4 Layers-CNN

In [ ]:
if model=='CNN4L':
    class CNN_4L(nn.Module):
        def __init__(self):
            super(CNN_4L,self).__init__()
            ### Patch Extraction n1=64 c=1 f1=9
            self.block1=nn.Conv2d(1,64,kernel_size=9,padding=2)
            ### Non-Linear Mapping n2=n1=64 f2=1
            self.block2=nn.Conv2d(64,64,kernel_size=1,padding=2)
            ### Image Reconstruction n3=n2=32 f3=5
            self.block3=nn.Conv2d(64,32,kernel_size=5,padding=2)
            ### Image Reconstruction n4=n=32 f3=3
            self.block4=nn.Conv2d(32,1,kernel_size=3,padding=1)
            self.RELU=nn.ReLU()
        def forward(self,out):
            out=self.block1(out)
            out=self.RELU(out)
            out=self.block2(out)
            out=self.RELU(out)
            out=self.block3(out)
            out=self.RELU(out)
            out=self.block4(out)
            return out

<h3>2.2 UNet
<img src="UNet.png"
 alt="SRCNN figure"
 style="float: left; margin-right: 10px;" />

In [9]:
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )
    def forward(self, x):
        return self.double_conv(x)

class Down(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )
    def forward(self, x):
        return self.maxpool_conv(x)

class Up(nn.Module):
    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        else:
            self.up = nn.ConvTranspose2d(in_channels // 2, in_channels // 2, kernel_size=2, stride=2)
        self.conv = DoubleConv(in_channels, out_channels)
    def forward(self, x1, x2):
        x1 = self.up(x1)
        # input is CHW
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]
        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)

class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)
    def forward(self, x):
        return self.conv(x)
    
if model=='UNet':
    class UNet(nn.Module):
        def __init__(self, n_channels, n_classes, bilinear=True):
            super(UNet, self).__init__()
            self.n_channels = n_channels
            self.n_classes = n_classes
            self.bilinear = bilinear
            self.inc = DoubleConv(n_channels, 64)
            self.down1 = Down(64, 128)
            self.down2 = Down(128, 256)
            self.down3 = Down(256, 512)
            self.down4 = Down(512, 512)
            self.up1 = Up(1024, 256, bilinear)
            self.up2 = Up(512, 128, bilinear)
            self.up3 = Up(256, 64, bilinear)
            self.up4 = Up(128, 64, bilinear)
            self.outc = OutConv(64, n_classes)
        def forward(self, x):
            x1 = self.inc(x)
            x2 = self.down1(x1)
            x3 = self.down2(x2)
            x4 = self.down3(x3)
            x5 = self.down4(x4)
            x = self.up1(x5, x4)
            x = self.up2(x, x3)
            x = self.up3(x, x2)
            x = self.up4(x, x1)
            logits = self.outc(x)
            return logits

NameError: name 'model' is not defined

<h3>2.3 DCGAN (Deep Convolution Generative Adversarial Network)

<h4> 2.3.1) Generator Network (UNet)

In [ ]:
if model=='DCGAN':
    class Generator(nn.Module):
        def __init__(self, n_channels, n_classes, bilinear=True):
            super(Generator, self).__init__()
            self.n_channels = n_channels
            self.n_classes = n_classes
            self.bilinear = bilinear
            self.inc = DoubleConv(n_channels, 64)
            self.down1 = Down(64, 128)
            self.down2 = Down(128, 256)
            self.down3 = Down(256, 512)
            self.down4 = Down(512, 512)
            self.up1 = Up(1024, 256, bilinear)
            self.up2 = Up(512, 128, bilinear)
            self.up3 = Up(256, 64, bilinear)
            self.up4 = Up(128, 64, bilinear) 
            self.outc = OutConv(64, n_classes)
        def forward(self, x):
            x1 = self.inc(x)
            x2 = self.down1(x1)
            x3 = self.down2(x2)
            x4 = self.down3(x3)
            x5 = self.down4(x4)
            x = self.up1(x5, x4)
            x = self.up2(x, x3)
            x = self.up3(x, x2)
            x = self.up4(x, x1)
            logits = self.outc(x)
            return logits

<h4>2.3.2) Discriminator Network (5 Layers CNN)

In [ ]:
if model=='DCGAN':
    class Discriminator(nn.Module):
        def __init__(self, ngpu):
            super(Discriminator, self).__init__()
            self.ngpu = ngpu
            self.main = nn.Sequential(
                # input is (nc) x 64 x 64
                nn.Conv2d(nc, ndf, 4, 3, 1, bias=False),
                nn.LeakyReLU(0.2, inplace=True),

                # state size. (ndf) x 32 x 32
                nn.Conv2d(ndf, ndf * 2, 4, 3, 1, bias=False),
                nn.BatchNorm2d(ndf * 2),
                nn.LeakyReLU(0.2, inplace=True),
                # state size. (ndf*2) x 16 x 16
                nn.Conv2d(ndf * 2, ndf * 4, 4, 3, 1, bias=False),
                nn.BatchNorm2d(ndf * 4),
                nn.LeakyReLU(0.2, inplace=True),
                # state size. (ndf*4) x 8 x 8
                nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 0, bias=False),
                nn.BatchNorm2d(ndf * 8),
                nn.LeakyReLU(0.2, inplace=True),
                # state size. (ndf*8) x 4 x 4
                nn.Conv2d(ndf * 8, 1, 4, 2, 0, bias=False),
                nn.Sigmoid()
            )

        def forward(self, input):
            return self.main(input)

<h2>
    <b>III. Network Training</b>
</h2>

<h3>3.1 Training Parameters Configuration

In [ ]:
# Number of training epochs
num_epochs = 5
# Learning rate for optimizers
lr = 0.0002
# Beta1 hyperparam for Adam optimizers
beta1 = 0.5

<h3>3.2 Resource Selection

In [ ]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'  # check whether a GPU is available

<h3>3.3 Loss Criterion

In [ ]:
# Initialize BCELoss function
if model=='DCGAN':
    criterion = nn.BCELoss()

    # Establish convention for real and fake labels during training
    real_label = 1
    fake_label = 0

    # Setup Adam optimizers for both G and D
    optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
    optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))
else:
# Initialize MSELoss function
    criterion=nn.MSELoss('Mean')
    optimizer=optim.SGD(model.parameters,lr=lr)

<h3>3.4 Train the network

In [ ]:
# Training Loop
# Lists to keep track of progress
img_list = []
G_losses = []
D_losses = []
iters = 0

print("Starting Training Loop...")
# For each epoch
for epoch in range(num_epochs):
    # For each batch in the dataloader
    for i, data in enumerate(train_loader, 0):

        x_data,y_data=data                         # undersampled and ground truth images
        
        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        ## Train with all-real batch
        netD.zero_grad()
        # Format batch
        real_cpu = y_data.to(device)
        b_size = real_cpu.size(0)
        label = torch.full((b_size,), real_label, device=device)
        
        # Forward pass real batch through D
        output = netD(real_cpu).view(-1)
        
        '''
        print('rcpu',real_cpu.shape)
        print('bsize',b_size)
        print('lab',label)
        
        print('Dout',netD(real_cpu).shape)
        print('Doutv',output.shape)
        '''
        
        # Calculate loss on all-real batch
        errD_real = criterion(output, label)
        # Calculate gradients for D in backward pass
        errD_real.backward()
        D_x = output.mean().item()

        ## Train with all-fake batch
        # Generate batch of latent vectors
        #noise = torch.randn(b_size, nz, 1, 1, device=device)
        noise = x_data.to(device)
        # Generate fake image batch with G
        fake = netG(noise)
        label.fill_(fake_label)
        #print((fake).shape)
        #break
        # Classify all fake batch with D
        output = netD(fake.detach()).view(-1)
        # Calculate D's loss on the all-fake batch
        errD_fake = criterion(output, label)
        # Calculate the gradients for this batch
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        # Add the gradients from the all-real and all-fake batches
        errD = errD_real + errD_fake
        # Update D
        optimizerD.step()
        
        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        netG.zero_grad()
        label.fill_(real_label)  # fake labels are real for generator cost
        # Since we just updated D, perform another forward pass of all-fake batch through D
        output = netD(fake).view(-1)
        # Calculate G's loss based on this output
        errG = criterion(output, label)
        # Calculate gradients for G
        errG.backward()
        D_G_z2 = output.mean().item()
        # Update G
        optimizerG.step()

        
        # Output training stats
        if i % 50 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, num_epochs, i, len(train_loader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))
        
        # Save Losses for plotting later
        G_losses.append(errG.item())
        D_losses.append(errD.item())
        
        '''
        # Check how the generator is doing by saving G's output on fixed_noise
        if (iters % 500 == 0) or ((epoch == num_epochs-1) and (i == len(train_loader)-1)):
            with torch.no_grad():
                fake = netG(fixed_noise).detach().cpu()
            img_list.append(vutils.make_grid(fake, padding=2, normalize=True))
        '''
        
        iters += 1
        #print('ydat:',y_data.shape)
        #print('fake:',fake.shape)
        #break

<h2>
    <b>IV. Inference(Image Reconstruction)</b>
</h2>

In [ ]:
file_path='C:/Users/Ersalina/NeuralComp_G42/saved_params'
model_path='C:/Users/Ersalina/NeuralComp_G42/saved_params/DCGAN_Gen01.pth'

test_file_path='C:/Users/Ersalina/NC_Final_Assignment/fastMRI/NC2019MRI/test'

In [ ]:
def kspace2real(data):
    return T.complex_abs(T.ifft2(T.to_tensor(data)))

In [ ]:
for fname in sorted(os.listdir(test_file_path)):
    with h5py.File(test_file_path+'/'+fname,  "r") as hf:
        volume_kspace_4af = hf['kspace_4af'][()]
        volume_kspace_8af = hf['kspace_8af'][()]
        print(f'{fname} 4af 8af',volume_kspace_4af.shape, volume_kspace_8af.shape)

In [ ]:
#print(kspace2real(volume_kspace_4af).shape)
inp=kspace2real(volume_kspace_4af)
Shape=inp.shape
inp=inp.view(Shape[0],1,Shape[1],Shape[2])
# Reconstruct the Image
#IR=model(inp[20])
fig = plt.figure(figsize=(15,10))
plt.imshow(inp[20][0], cmap='gray')
plt.axis('off')


inp2=torch.randn(1,1,2,2)
out=model(inp)
print(out.shape)